# Assignment 1 - 2D Particle Simulator

Zac Keskin - Techniques of High Performance

This notebook contains a simulation of particle-particle interactions in a 2D region bounded by [0,1] in each dimension, where the interations are defined by the potential felt by particle $i$ due to particle $j$:
$$
V_{ij} = -\, \frac{k_{j}}{2} \, \log \,\left|(x_i-x_j)^2 + (y_i - y_j)^2 \right|
$$ <br>
Where $k_i$ is some scaing constant associated with each particle (e.g. 'charge') and where we have taken the Euclidean norm in 2D to define the distance between particles. <BR><BR>
    
Key design decisions were made in order to in ensure the high performance of the simulation. 
- Primarily, we present the simulation graphically by plotting the potential on a discrete mesh, where the potential at each element of the mesh is calculated in parallel using PyOpenCL.
- Secondly, we parallelise across all particles the calculation of force, by summing the contributions from the potential of all other particles, resulting in $O(N)$ efficiency
- We then perform vector operations in numpy to efficiently update velocities and positions accordingly, using the leapfrog algorithm to integrate through time 

<BR><BR>
    
There are a number of parameters defining the simulation which can be adjusted to investigate different behaviours. These are described below: <br>

- `dtype` = Define whether to use single or double precision (chose `'64'` or `'32'`)
- `device` = Choose whether to use 'GPU' or 'CPU'. If 'GPU' is chosen and none is available, the CPU will be used 
- `m` - Define the fineness of the discretised mesh
- `N` - The number of particles
- `k` - A coefficient defining the strength of the potential between particles. Currently applies the same k to all particles (int, default 1)
- `reflect` - Define whether the particles should reflect at the boundary or not (Boolean, default `False`)

You can also change the line in order to initialise the particle positions:`particles = equal(N)`

Use `particles = gaussian(N)` or `particles = uniform(N)` to initialise the particles under normal or uniform random distributions respectively.

____

### Prepare the simulation by defining chosen parameters

In [ ]:
# Parameters
dtype = '32'   
device = 'GPU' 
N = 24
m = 100
k = 1   # We presently give all particles k=1
r = 0.3 # Radius of initial circle, where equal() used
reflect = False

### Installing Prerequisites  
If running on Azure, the default VM does not have PyOpenCL or VTK installed, so the first cell should be run to install these. 
- Please note, this can be quite slow, **so only run this cell if necessary**

In [ ]:
!conda install -c conda-forge --yes --verbose pyopencl pocl

In [ ]:
import pyopencl as cl
import numpy as np
from numpy.linalg import norm
from math import pi, cos, sin
import random
import os

### Including the Timer class 
This was provided by Dr Betcke, and will be used to measure the time taken to calculate the potential field

In [ ]:
import time

class Timer:    
    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, *args):
        self.end = time.time()
        self.interval = self.end - self.start

### We create a particle class to hold the data on each particle
This is used in the initialisation stage, for the sake of simplicity, but was found to be inefficient as an approach for parallelising the implementation, since the properties must be sent to device memory using numpy arrays. 

In [ ]:
class Particle():

    def __init__(self, x, y, k = 1):
        self.x = x
        self.y = y
        self.k = k
        

### We create three functions to initialise the positions of all N particles
When initialising the particles, we select one of the three functions and return a list of particle objects distributed as described. From this initial position we can progress our simulation through time.

In [ ]:
# Create Particles equally spaced across circle radius = r 
def equal(N):
    return [Particle(x = 0.5 + cos(2 * pi/N * n)*r,
                     y = 0.5 + sin(2 * pi/N * n)*r,
                     k = k)  for n in range(0,N)]

# Create Particles uniformly distributed
def uniform(N):
    return [Particle(x = random.uniform(0,1),
                     y = random.uniform(0,1),
                     k = k)  for n in range(0,N)]

# Create Particles normally distributed
def gaussian(N): 
    return [Particle(x = random.gauss(0.5,0.1),
                     y = random.uniform(0.5,0.1),
                     k = k)  for n in range(0,N)]

### Defining functions to return the potential field, given a set of particle coordinates
We describe both a simple sequential implementation and also a parallelised implementation, and will later compare the performance in calculating the potential field, which is then used to plot our system of particles

In [ ]:
# Sequential naive-Python implementation of Potential field calculation
def get_potential(x,y,particles,ks):
    # Potential is k*log(distance to particle) for each particles
    N  = len(particles)
    xs = np.array(x for p in particles)
    ys = np.array(y for p in particles)

    xjs = particles[:,0]
    yjs = particles[:,1]

    # Sum the contributions from each particle   
    v=0
    for j in range(len(particles)):
        v = v + ks[j] * np.log( ( (x-xjs[j])**2 + (y-yjs[j])**2 )**0.5 )
    return -v 


# Parallel, PyOpenCL implementation of Potential field calculation
def get_potential_OpenCL(xx,yy,particles):
    # Prepare tensors for use on device
    xjs = particles[:,0].astype(dfloat)
    yjs = particles[:,1].astype(dfloat)
    V = np.empty_like(xx).astype(dfloat)

    n_particles = np.array([len(xjs)]).astype(np.int32)
    # Create buffers on device memory
    xx_buffer = cl.Buffer(context, mem_flags.READ_ONLY | mem_flags.COPY_HOST_PTR, hostbuf= xx)
    yy_buffer = cl.Buffer(context, mem_flags.READ_ONLY | mem_flags.COPY_HOST_PTR, hostbuf= yy)
    xjs_buffer = cl.Buffer(context, mem_flags.READ_ONLY | mem_flags.COPY_HOST_PTR, hostbuf= xjs)
    yjs_buffer = cl.Buffer(context, mem_flags.READ_ONLY | mem_flags.COPY_HOST_PTR, hostbuf= yjs)
    ks_buffer = cl.Buffer(context, mem_flags.READ_ONLY | mem_flags.COPY_HOST_PTR, hostbuf= ks)
    particles_buffer = cl.Buffer(context, mem_flags.READ_ONLY | mem_flags.COPY_HOST_PTR, hostbuf= n_particles)
    out_buffer = cl.Buffer(context, mem_flags.WRITE_ONLY, V.nbytes)
    kernel = """   
            __kernel void potential(const __global float *xs, 
                                    const __global float *ys,
                                    const __global float *xjs, 
                                    const __global float *yjs,
                                    const __global int *n_particles,
                                          __global float *Vs)
            {
                const int gid = get_global_id(0);
                
                float V = 0.0;
                float x = xs[gid]; 
                float y = ys[gid]; 
                int N = n_particles[0];
                
                for (int j=0; j<N; j++){
                    V += 0.5*  log(    
                                pown(x - xjs[j], 2) 
                             +  pown(y - yjs[j], 2) 
                                );           
        
                // Store the result
                Vs[gid] = -V;
                } 
            }

             """
    
    if dtype == '64':
        kernel = kernel.replace("float ","double ")
    # Run OpenCL Program 
    program = cl.Program(context, kernel).build()
    program.potential(queue, ((m)*(m),), None, xx_buffer, yy_buffer, xjs_buffer, yjs_buffer, particles_buffer, out_buffer)
    cl.enqueue_copy(queue, V, out_buffer)
    return V

### Defining the parallel implementation to return the force on each particle due to the other particles
We can use the derivative of the inter-particle potential to calculate the interparticle force, and sum the contributions from all other particles. This is done for each particle in parallel using PyOpenCL

In [ ]:
def get_forces_OpenCL(particles,ks):

    # Create numpy objects to define memory requirements
    forces = np.empty((N,2)).astype(dfloat)
    xjs = positions[:,0].flatten().astype(dfloat)
    yjs = positions[:,1].flatten().astype(dfloat)
    
    n_particles = np.array([len(xjs)]).astype(np.int32)
    # Create OpenCL input buffers to store np arrays
    xjs_buffer = cl.Buffer(context, mem_flags.READ_ONLY | mem_flags.COPY_HOST_PTR, hostbuf= xjs)
    yjs_buffer = cl.Buffer(context, mem_flags.READ_ONLY | mem_flags.COPY_HOST_PTR, hostbuf= yjs)
    ks_buffer = cl.Buffer(context, mem_flags.READ_ONLY | mem_flags.COPY_HOST_PTR, hostbuf= ks)
    particles_buffer = cl.Buffer(context, mem_flags.READ_ONLY | mem_flags.COPY_HOST_PTR, hostbuf= n_particles)
    forces_buffer = cl.Buffer(context, mem_flags.WRITE_ONLY, forces.nbytes)
    
    # Force on each particle is 
    forces_kernel = """   
            __kernel void force(   const __global float *xjs, 
                                    const __global float *yjs,
                                    const __global float *ks,
                                    const __global int *n_particles,
                                          __global float *forces)
            {
                const int gid = get_global_id(0);
                
                float Fx = 0.0;
                float Fy = 0.0;
                int N = n_particles[0];
                
                for (int j=0; j<N  ; j++){
                    float xj = xjs[j];
                    float yj = yjs[j];
                    
                    if (j != gid){
                        Fx += -ks[gid] * (xj-xjs[gid]) /( (xjs[gid]-xj)*(xjs[gid]-xj) + (yjs[gid]-yj)*(yjs[gid]-yj) );
                        Fy += -ks[gid] * (yj-yjs[gid]) /( (xjs[gid]-xj)*(xjs[gid]-xj) + (yjs[gid]-yj)*(yjs[gid]-yj) );           
                    };

                forces[gid*2] = Fx;
                forces[gid*2+1] = Fy;          
                } 
            }

             """
    if dtype == '64':
        forces_kernel = forces_kernel.replace('float ','double ')
    
    # Run progream
    program = cl.Program(context, forces_kernel).build()
    program.force(queue, ((N)*(N),(N)*(N)), None, xjs_buffer, yjs_buffer, ks_buffer, particles_buffer, forces_buffer)
    cl.enqueue_copy(queue, forces, forces_buffer)
    return forces

_______

## Preparing the Simulation

### Create a new OpenCL Context
We prepare to parallelise the problem and solve for each meshgrid element at once, by using the GPU (if available)

In [ ]:
# Create new OpenCL Context
platforms = cl.get_platforms()
if device == 'GPU' and len(platforms[0].get_devices(device_type=cl.device_type.GPU)) > 0:
    devices = platforms[0].get_devices(device_type=cl.device_type.GPU)
else:
    devices = platforms[0].get_devices(device_type=cl.device_type.CPU)

print('Operating on ' + str(devices[0].max_compute_units) + ' threads', '\n' )

# Check that the device can handle double precision
if devices[0].double_fp_config == 0 and dtype == '64':
    print('Your selected device can not use double precision decimals. \n ' +
            'Running simulation using single precision floats \n')
    dtype = 32


# Configure type for np arrays
if dtype == '64':
    dfloat = np.float64
else:
    dtype = '32'
    dfloat = np.float32
    
    
context = cl.Context(devices=devices)
queue = cl.CommandQueue(context) 
mem_flags = cl.mem_flags

### Discretising the domain, using numpy to return the meshgrid
m represents the number of points in each discretised axis of the unit box. Therefore the number of elements for which the potential must be calculated scales with $O(m^2)$

In [ ]:
# Discretise the unit box using a numpy meshgrid:
x = np.linspace(0, 1, m).astype(dfloat)
y = np.linspace(0, 1, m).astype(dfloat)
xx, yy = np.meshgrid(x,y)


____

## Running the Simulation

### Initialising the Particle Positions

In [ ]:
# Define Particles
particles = equal(N)
# Use np arrays for simpler interation
positions = np.array([[p.x,p.y] for p in particles]).astype(dfloat)  # list containing the [x,y] coordinates
ks = np.array([k for p in particles]).astype(dfloat) # list containing k for each particle

### Initialising the Potential Field 
We compare the sequential python implementation with the parallel PyOpenCL implementation to calcultate the potential in each gridspace

In [ ]:
with Timer() as t:
    potential_field =  get_potential(xx[:,None], yy[None,:], positions, ks)[0]
print('Python Potential Field Calculation on ' + str(m*m) + ' gridspaces: '  + str(round(t.interval,4)) + 's')

In [ ]:
with Timer() as t:
    potential_field = get_potential_OpenCL(xx, yy, positions).astype(dfloat)
print('OpenCL Potential Field Calculation on ' + str(m*m) + ' gridspaces: '  + str(round(t.interval,4)) + 's')
print('Parallel Gridspace calculations per Second: ' + str(round(m*m/t.interval)) ,'\n')

We note a marked improvement in the required time to calculate the potential field. This simulation simply would not be feasible without parallelisation.

### Initialising the Force on each particle
We use the parallel computation defined above to return the force felt by each particle, and present key timing metrics.

In [ ]:
with Timer() as t:
    forces = get_forces_OpenCL(positions,ks).astype(dfloat)
print('OpenCL Calculation of ' + str(N*(N-1)) + ' particle interactions: ' + str(round(t.interval,4)) + 's')
print('Particle-Particle Interations per Second: ' + str(round(N*(N-1)/t.interval)))

We note that the performance gains increase with larger simulations. With N=200 we find: 

<br> Time taken for OpenCL calculation of 39,800 particle interactions: **0.1785s** <br>
Particle-Particle Interations per Second: **222,909** <br><br>

We note that this metric instead include the update of the positions and velocities as well as the force update, to measure the time for a complete particle-particle interaction; however the position and velocity updates are simply numpy vector operations, so the force update dominates the size of the calculation. Further, we find the initial computation to be considerably slower than those within the later steps of the loop, so the above figures are within approximate reason.

### Initialising the Velocities of each particle
The Leapfrog Method can be implemented by performing an inital Forward Euler step to calculate the inital velocities of each particle due to the forces initialised above 
$$
v_i^{(1/2)} = v_i^{(0)} + \frac{1}{2}\Delta t F(x_i^{(0)}).
$$

In [ ]:
dt = 0.001
velocities = (0.5 * dt * forces)

### Initialise the Animation

We use Matplotlib's animate functionality to update the 2D imshow plot as we increment time, calculating new positions which then cause new forces on the particles, resulting in new velocities subject to
\begin{align}
v_i^{(\ell + \frac{3}{2})} &= v_i^{(\ell + \frac{1}{2})} + \Delta t F(x_i^{\ell})\nonumber
\end{align}
These velocities are then used to update the positions of each particle in the next loop of the simulation, according to:
\begin{align}
x_i^{(\ell + 1)} &= x_i^{(\ell)} + \Delta t v_i^{(\ell + \frac{1}{2})}\nonumber\\
\end{align}

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

fig = plt.figure()
im = plt.imshow(potential_field, interpolation='bicubic',cmap='Blues',
                extent=[0, 1, 0, 1], animated=True, vmax = np.max(potential_field))
plt.xlabel('x')
plt.ylabel('y')
plt.title('Potential, V(x,y)')


In [ ]:
%%capture
def animate(*args):
    global dt, velocities, positions, forces
    # Update Positions
    positions = positions + dt * velocities

    # Update Forces in parallel using OpenCL
    forces = get_forces_OpenCL(positions, ks)

    # Update Velocities
    velocities = velocities + dt * forces 
    # Include reflection along axes for particle collisions with border
    elasticity = 0.9
    if reflect == True:
        velocities[np.logical_or(positions>1, positions<0)] =  - elasticity * velocities[np.logical_or(positions>1, positions<0)]
    
    
    # Update potential field for plotting
    potential_field = get_potential_OpenCL(xx,yy,positions)
    im.set_array(potential_field)
    
    return im,

anim = animation.FuncAnimation(fig, animate, frames=500, interval=1)

In [ ]:
# Show the Animation
%matplotlib inline
HTML(anim.to_jshtml())

### Creating a Surface Plot
We also present a surface plot of the potential field as a second, interesting way to visualise the simulation

In [ ]:
# Reinitialise the positions
particles = equal(N)
# Use np arrays for simpler interation
positions = np.array([[p.x,p.y] for p in particles]).astype(dfloat)  # list containing the [x,y] coordinates
ks = np.array([k for p in particles]).astype(dfloat) # list containing k for each particle

# Reinitialise the potentials
potential_field = get_potential_OpenCL(xx, yy, positions)

In [ ]:
%matplotlib inline
from mpl_toolkits.mplot3d import axes3d
fig2 = plt.figure()

ax = fig2.add_subplot(111, projection="3d")
ax.plot_surface(xx, yy, potential_field, cmap="Blues", rstride=1, cstride=1, alpha=0.8) #, lw=0.8

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('V(x,y)')

plt.xlabel('x')
plt.ylabel('y')
plt.title('Potential, V(x,y)')

plt.show()